In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
import sys, os

In [ ]:
from datawand.parametrization import ParamHelper
ph = ParamHelper("../pipelines/GcnProject.json",sys.argv)

# 1. Collect accuracy results

In [ ]:
input_prefix = ph.get("experiment_dir")

In [ ]:
top_k_vals ,time_frame_vals, cut_ratio_vals = ph.get("top_k_vals"), ph.get("time_frame_vals"), ph.get("cut_ratio_vals")

In [ ]:
accuracies = []
for k in top_k_vals:
    for delta in time_frame_vals:
        for cut in cut_ratio_vals:
            with open("%s/data/k%i_t%i_r%.2f/acc.csv" % (input_prefix,k,delta,cut)) as f:
                accuracies.append([k, delta, cut] + f.read().split(";"))

In [ ]:
accuracy_df = pd.DataFrame(accuracies,columns=["top_k","time_frame","train_ratio","train_acc","test_acc","val_acc"])
for feat in ["train_acc","test_acc","val_acc"]:
    accuracy_df[feat] = accuracy_df[feat].astype("f")

In [ ]:
accuracy_df

# 2. Visualization

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
cut_1 = accuracy_df[accuracy_df["train_ratio"]==0.5]
cut_2 = accuracy_df[accuracy_df["train_ratio"]==0.7]

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 5), sharex=True)

f1=sns.factorplot(x="time_frame", y="test_acc", hue="top_k", data=cut_1, size=4, kind="bar", palette="muted", ax=ax1)
f2=sns.factorplot(x="time_frame", y="train_acc", hue="top_k", data=cut_1, size=4, kind="bar", palette="muted", ax=ax2)
f3=sns.factorplot(x="time_frame", y="val_acc", hue="top_k", data=cut_1, size=4, kind="bar", palette="muted", ax=ax3)

plt.close(f1.fig)
plt.close(f2.fig)
plt.close(f3.fig)

In [ ]:
tf_60 = accuracy_df[accuracy_df["time_frame"]==60]

In [ ]:
tf_60